In [1]:
import sys
import os
sys.path.append(os.path.split(os.path.split(sys.path[0])[0])[0])

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
import plotly_template as pt
from functions import emissions, func, mp_funcs
import definitions
import pandas as pd
import pickle
import math
import numpy as np

# pio.renderers.default = "browser"

# Generating Iterations of the Heatmaps

### Creating the emissions plot function

### File locations and constants

In [3]:
DATA_FOLDER = '2020-02-13_2'
DATA_DAY = '2020-02-13'
DATA_DIR = r'Z:\6 ACTION Project\SUMO\Sim_Output_Data\7-28-2020' 
# DATA_DIR_FULL_PATH = os.path.join(definitions.DATA_DIR, DATA_FOLDER)
DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, DATA_FOLDER)
RAW_DATA_FILE = 'data.csv'
RAW_DATA_SUMMARY = 'data_summary.csv'

### Bining the Emissions Data

The time is >=time[i] & < time[i+1]

In [4]:
resample_period = '120T' # aka the bin size
bin_size = 5

In [5]:
df = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
#df['timestep_time'] = pd.to_datetime(df['timestep_time'])


start_time = "02/12/2020 23:00:00.000"

if df['timestep_time'].dtype.name == 'float64':
    """ These don't work on Windows. Take that back. They do not work in scientifc mode with PyCharm """
    # print("Calling mp apply")
    df.loc[:, 'timestep_time'] = mp_funcs.apply_by_multiprocessing(df['timestep_time'], func, start_time=start_time)
    # raw_data_df.loc[:, 'timestep_time'] = pd.to_datetime(raw_data_df.loc[:, 'timestep_time'])
    # raw_data_df['timestep_time'] = raw_data_df['timestep_time'].apply(func=func)
    df.loc[:, 'timestep_time'] = pd.to_datetime(df.loc[:, 'timestep_time'])
    print("Saving to .csv")
    df.to_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE))
    
elif df['timestep_time'].dtype.name == 'object':
    df.loc[:, 'timestep_time'] = pd.to_datetime(df.loc[:, 'timestep_time'])

c:\users\spraychamber\us69-research-paper-plotting\venv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Saving to .csv


In [12]:
mask = df['vehicle_fuel'] <= 0.000076
df.loc[mask, 'vehicle_fuel'] = 0.000077

emissions.bin_2D_sum(emissions_df=df, bin_column=['vehicle_fuel'], resample_period=resample_period, bin_size=5, processor_num=1)

func:bin_2D_sum took: 5.78083062171936 sec


(None, 5.78083062171936)

### Grabbing the Data

In [7]:
binned_emissions_dict = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

### Plotting the Data

#### Plot Constants

In [8]:
plot_radius = 10
mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"

colorscale = [
              [0.0, "rgba(0, 255, 204, 0)"],
              [0.2, "rgb(0, 255, 51)"],
              [0.4, "rgb(204, 255, 0)"],
              [0.6, "rgb(255, 204, 51)"],
              [0.8, "rgb(255, 102, 51)"],
              [1.0, "rgb(204,0,0)"],
            ]

color_axis = dict(
            cmin=0,
            cmax=None,
            showscale=True,
            colorscale=colorscale,
            colorbar=dict(
                outlinecolor="black",
                outlinewidth=2,
                ticks="outside",
                # tickfont=pt.font_dict,
                tickvals=None, # tickvalues,
                ticktext=None,
                # title="[$gal/25m^2$]<br /> <br />",
                title="[gal/25m^2] <br /> <br />",
                tickformat="e",
                # titlefont=pt.font_dict,
            )
)

In [9]:
plot_time_list = [pd.to_datetime('2020-02-13T05:00:00'), pd.to_datetime('2020-02-13T07:00:00'), pd.to_datetime('2020-02-13T15:00:00'), pd.to_datetime('2020-02-13T17:00:00')]
plot_titles = ['5AM - 7AM', '7AM - 9AM', '3PM - 5PM', '5PM - 7PM']
plot_titles = [plot_titles[i] + f"<br /> Total Fuel Consumed: {round(binned_emissions_dict[time]['interval_sum'], 2)} gallons" for i, time in enumerate(plot_time_list)]
num_plots = len(plot_time_list)
fig = make_subplots(rows=1, cols=num_plots,
                    specs=[[{'type': 'mapbox', }] * num_plots],
                    subplot_titles=plot_titles,
                    column_widths=[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots,
                    horizontal_spacing=0.05,)

mapbox_dict = {'mapbox': dict(accesstoken=mapbox_key,
                                bearing=0,
                                style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
                                center=go.layout.mapbox.Center(
                                    lat=33.126275,
                                    lon=-87.54891
                                ),
                                pitch=0,
                                zoom=14.35),}

max_value = -20
for i, time in enumerate(plot_time_list):
    
    z_data = binned_emissions_dict[time]['data']
    lat_edges = binned_emissions_dict[time]['lat']
    lon_edges = binned_emissions_dict[time]['lon']
    max_value = binned_emissions_dict[time]['max_value'] if binned_emissions_dict[time]['max_value'] > max_value else max_value
    
    fig.add_trace(go.Densitymapbox(lat=lat_edges,
                                    lon=lon_edges,
                                    z=z_data,
                                    # customdata=summed_array[0][2],
                                    # hovertemplate='%{customdata}' if sum else None,
                                    radius=plot_radius,
                                    hoverinfo='z',
                                    coloraxis="coloraxis",
                                    ),
                    row=1,
                    col=i+1)
    
    mapbox_dict['mapbox'+ str(i+2)] =  mapbox_dict['mapbox']
    
color_axis['cmax'] = math.ceil(max_value * 1000) / 1000

len_of_max = len(str(color_axis['cmax']))
# tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max-1), 7), 3)
tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max), 7), 3)
ticktext = [str(val) for val in tickvalues]

color_axis['colorbar']['tickvals'] = tickvalues
color_axis['colorbar']['ticktext'] = ticktext

In [10]:
fig.update_layout(mapbox_dict)

fig.update_layout(
    title={"text": "Fuel Consumption", 'yanchor': 'middle', 'xanchor': 'center'},
    hovermode='closest',
#     mapbox=dict(
#         accesstoken=mapbox_key,
#         bearing=0,
#         style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
#         center=go.layout.mapbox.Center(
#             lat=33.12627,
#             lon=-87.54891
#         ),
#         pitch=0,
#         zoom=14.1,
#     ),
    coloraxis=color_axis
)
fig.update_layout(template='simple_white', font_family='helvetica', font_size=16,) #height=600, width=1200,)

fig.show('browser')
# pio.write_json(fig, os.path.join(definitions.ROOT, 'raw_plots', 'mapbox.plotly'))